In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_error, f1_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm

C:\Users\avitr\AppData\Roaming\Python\Python39\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
crm_df = pd.read_csv("CRM_DATA.csv")
ga4_df = pd.read_csv("GA4DATA.csv")
ga4_df.drop(columns=['user_id'],inplace=True)

In [3]:
mapping_df = ga4_df.copy()

In [4]:
ga4_df.shape

(90323, 65)

In [5]:
# Find and assign ReferenceID values to instances of 'user_pseudo_id'
for _, row in tqdm(mapping_df.iterrows()):
    user_id = row['user_pseudo_id']
    ref_id = row['ReferenceID']
    ga4_df.loc[ga4_df['user_pseudo_id'] == user_id, 'ReferenceID'] = ref_id

0it [00:00, ?it/s]

In [ ]:
# leap_compain = name

In [18]:
merged_df = pd.merge(left=crm_df,right=ga4_df,left_on='Leap_Reference_ID',right_on='ReferenceID')

In [19]:
merged_df.shape

(24496, 81)

In [20]:
merged_df.drop(columns=['Leap_Campaign','Leap_Source','Leap_Medium','ReferenceID'],inplace=True)

In [21]:
null_val = ['(not set)','(none)']

In [245]:
merged_df_backup = merged_df.copy()

In [246]:
# merged_df = merged_df_backup.copy()

In [247]:
cols = ['PageElement','LeadManager_CustomerNote','LinkElement','PageElement','LinkURL','LinkSelector','GoogleClickIdentifier',
'mobile_os_hardware_model','mobile_os_hardware_model',
'vendor_id',
'advertising_id',
'time_zone_offset_seconds',
'browser',
'browser_version',
'hostname_1',
# 'manual_c',
'dclid',
'srsltid',]

In [248]:
merged_df_backup.drop(cols,axis=1, inplace=True)

In [249]:
# drop_cols = ['AAAMember',
#  'BatchOrderingID',
#  'BatchPageID',
#  'EngagementTime_msec',
#  'GASessionNumber',
#  'IgnoreReferrer',
# #  'IsShipmentBooked',
#  'LeadManager_DestinationCountry',
#  'LeadManager_DestinationState',
# #  'Leap_Reference_ID',
#  'LinkClasses',
#  'LinkText',
#  'SessionEngaged',
#  'UniqueSearchTerm',
#  'browser_1',
#  'continent',
#  'country',
#  'gclid',
#  'language',
#  'medium',
#  'metro',
#  'mobile_brand_name',
#  'mobile_marketing_name',
#  'mobile_model_name',
#  'name',
#  'operating_system',
#  'operating_system_version',
#  'region',
#  'source',
#  'sub_continent']

In [250]:
# col_drop = ['LeadManager_OriginCity',        
# 'LeadManager_OriginCounty',           
# 'LeadManager_OriginState',             
# 'LeadManager_DestinationCity',     
# 'LeadManager_DestinationCounty',   
# 'city','stream_id','hostname','browser_version_1','user_first_touch_timestamp','GASessionID','PageReferrer','PageTitle','LeadManager_OriginZip','event_timestamp','event_date',
# 'PageLocation','LeadManager_DestinationZip','ReferenceID','user_pseudo_id','LeadManager_OriginCountry','platform' ,'is_limited_ad_tracking','EngagedSessionEvent','LeadManager_OriginCountry']         


In [251]:
# merged_df.drop(drop_cols,axis=1, inplace=True)

In [252]:
# cols = ['event_date', 'stream_id', 'browser_version_1' ,'GASessionID', 'PageLocation' ,'event_timestamp' ,'event_date']

In [253]:
# merged_df.drop(cols,axis=1,inplace=True)

In [254]:
def get_fill_data(df,userID,col):
    
    val = [i for i in df.groupby('user_pseudo_id')[col].unique()[userID] if not pd.isna(i)]
    
    if len(val):
        return val[0]
    else:
        return np.nan

In [255]:
cols = set(merged_df_backup) - {'IsShipmentBooked', 'Leap_Reference_ID', 'user_pseudo_id'}

In [256]:
# https://devskrol.com/2020/08/09/best-way-to-impute-nan-within-groups-mean-mode/
for col in tqdm(cols):
    merged_df_backup[col] = merged_df_backup.groupby(['user_pseudo_id'], sort=False)[col].apply(lambda x: x.fillna(x.mode().iloc[0] if len(x.mode()) else np.nan))

  0%|          | 0/59 [00:00<?, ?it/s]

In [257]:
# for indx, row in tqdm(merged_df.iterrows()):
#     for col_name in cols:
#         val = row[col_name]
#         user_id = row['user_pseudo_id']
#         if pd.isna(val):
#             rplace = get_fill_data(merged_df_backup,user_id,col_name)
#             merged_df_backup.at[indx, col_name] = rplace

In [258]:
na = merged_df_backup.isna().sum()*100/len(merged_df)

In [259]:
drop_col = []
print('********* Columns to be dropped ****************\n')
for i,col in zip(na,na.index):
    if i >=30:
        print(i,'-->>',col)
        drop_col.append(col)

********* Columns to be dropped ****************

55.776453298497714 -->> AAAMember
40.443337687785764 -->> IgnoreReferrer
41.516982364467665 -->> LinkClasses
38.63079686479425 -->> LinkText
39.50849118223383 -->> SearchTerm
66.14957544088831 -->> BatchOrderingID
66.14957544088831 -->> BatchPageID
39.50849118223383 -->> UniqueSearchTerm
88.52057478772045 -->> mobile_marketing_name
100.0 -->> manual_campaign_id
90.16165904637492 -->> manual_content
49.79180274330503 -->> gclid


In [260]:
merged_df_backup.drop(drop_col,axis=1,inplace=True)

In [261]:
merged_df_backup.shape

(24496, 50)

In [264]:
merged_df_backup.dropna(inplace=True)

In [265]:
merged_df_backup.shape

(18905, 50)

In [ ]:
drop_col = ['stream_id','hostname','browser_version_1','user_first_touch_timestamp','GASessionID','PageReferrer','PageTitle','LeadManager_OriginZip','event_timestamp','event_date',
'PageLocation','LeadManager_DestinationZip','ReferenceID','user_pseudo_id']

In [ ]:
merged_df.drop(drop_col,axis=1,inplace=True)

In [ ]:
merged_df.nunique()

In [ ]:
drop_col = ['LeadManager_OriginCountry','platform' ,'is_limited_ad_tracking','EngagedSessionEvent','LeadManager_OriginCountry']         

In [ ]:
merged_df.drop(drop_col,axis=1,inplace=True)

In [ ]:
merged_df.drop_duplicates(inplace=True)

In [ ]:
merged_df.reset_index(drop=True,inplace=True)

In [ ]:
merged_df_copy = merged_df.copy()

In [ ]:
merged_df_copy.drop('Leap_Reference_ID',axis=1,inplace=True)

In [ ]:
col_drop = ['LeadManager_OriginCity',        
'LeadManager_OriginCounty',           
'LeadManager_OriginState',             
'LeadManager_DestinationCity',     
'LeadManager_DestinationCounty',   
'city']

In [ ]:
merged_df_copy.drop(col_drop,axis=1,inplace=True)

In [ ]:
merged_df_copy.shape

In [ ]:
merged_df['IsShipmentBooked'].value_counts(normalize=True)*100

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.countplot(data=merged_df,x='IsShipmentBooked') # SMOTE

In [ ]:
merged_df

In [ ]:
for col in merged_df_copy:
    merged_df_copy = merged_df_copy[~merged_df_copy[col].isin(null_val)]

In [ ]:
merged_df_copy.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
column_to_scale = merged_df_copy['EngagementTime_msec'].values.reshape(-1, 1)
scaled_column = scaler.fit_transform(column_to_scale)
merged_df_copy['EngagementTime_msec'] = scaled_column

In [ ]:
merged_df_copy.dropna(inplace=True)

In [ ]:
X = merged_df_copy.drop('IsShipmentBooked',axis=1)
y = merged_df_copy['IsShipmentBooked']

In [ ]:
X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn import tree
from sklearn import ensemble
from sklearn import linear_model
from sklearn import gaussian_process
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import svm
from sklearn import model_selection
from sklearn import discriminant_analysis
from xgboost.sklearn import XGBClassifier

In [ ]:
MLA = [
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.RandomForestClassifier(n_estimators = 100, random_state = 0),
    
    gaussian_process.GaussianProcessClassifier(),
    
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    neighbors.KNeighborsClassifier(),
    
    svm.SVC(probability=True),
    svm.NuSVC(probability = True),
    svm.LinearSVC(),
    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),
    
    XGBClassifier(), 
    
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron()
]

In [ ]:
MLA_columns = ['MLA_names', 'MLA_parameters', 'MLA_Train_Accuracy_Mean'
               ,'MLA_Test_Accuracy_Mean', 'MLA_Test_Accuracy_3*STD', 
               'MLA_Time']

MLA_compare = pd.DataFrame(columns = MLA_columns)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
MLA_Predict = y_test
row_index = 0
for alg in tqdm(MLA):
    try:
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, 'MLA_names'] = MLA_name
        MLA_compare.loc[row_index, 'MLA_parameters'] = str(alg.get_params())
        cv_results = model_selection.cross_validate(alg, X_train, y_train, cv=3, return_train_score = True)
        MLA_compare.loc[row_index, 'MLA_Time'] = cv_results['fit_time'].mean()
        MLA_compare.loc[row_index, 'MLA_Train_Accuracy_Mean'] = cv_results["train_score"].mean()
        MLA_compare.loc[row_index, 'MLA_Test_Accuracy_Mean'] = cv_results['test_score'].mean()
        MLA_compare.loc[row_index, 'MLA_Test_Accuracy_3*STD'] = cv_results['test_score'].std()*3

        alg.fit(X_train, y_train)
        MLA_Predict[MLA_name] = alg.predict(X_test)
        row_index += 1
        print(".", end="")
    except:
        print(type(alg).__name__)
MLA_compare.sort_values(by = 'MLA_Test_Accuracy_Mean', ascending = False, inplace = True)
MLA_compare

In [ ]:
clf =  tree.DecisionTreeClassifier()

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
merged_df.shape

In [ ]:
merged_df_copy.shape

In [ ]:
y.value_counts()

In [ ]:
d = clf.predict_proba(X_test)

In [ ]:
for i in d:
    print(i)

In [ ]:
d[125]

In [ ]:
d[:,0]

# Get CRM Data

In [ ]:
# Get CRM Data
crm_df = pd.read_csv("MarketingData_01_16_2024.csv")
def convert_columns_to_int(dataframe, columns_to_convert):
    for column in columns_to_convert:
        dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce')
    return dataframe

int_columns = ['LeadManager_LeadID', 'LeadManager_AgentPrimary', 'LeadManager_EmployerZip',
               'LeadManager_AgentPrimary', 'LeadManager_AgentDestination', 'LeadManager_AgentOrigin',
               'LeadManager_OriginZip', 'LeadManager_DestinationZip', 'LeadManager_WeightPerRoom',
               'LeadManager_MoveTypeID', 'LeadManager_TotalRooms', 'LeadManager_Bedrooms', 
               'LeadManager_Appliances', 'LeadManager_Distance', 'LeadManager_Weight', 
               'LeadManager_Linehaul', 'LeadManager_StairFlights', 'LeadManager_StorageDuration',
               'LeadManager_OpenLead', 'LeadManager_NumberOfAttempts', 'LeadManager_Vehicles',
               'LeadManager_SmartMoveVaults', 'LeadManager_EstimatedTransitDays',
               'LeadManager_OriginCountyID', 'ACM_CustomerID', 'Order_OrdBookingAgencyCode',
               'Order_OrdPVO'
]

def convert_columns_to_date(dataframe, columns_to_convert):
    for column in columns_to_convert:
        dataframe[column] = pd.to_datetime(dataframe[column], errors='coerce')
    return dataframe

date_columns = ['Leap_LastUpdDateTime', 'LeadManager_EstimatedPackDate', 'LeadManager_EstimatedLoadDate',
                'LeadManager_EstimatedDeliveryDate', 'LeadManager_DateEstimateScheduled', 'LeadManager_DateCreated',
                'LeadManager_DateBooked', 'LeadManager_DateCSRAssigned', 'LeadManager_DatePrimaryAgentAssigned',
                'LeadManager_DateOriginAgentAssigned', 'LeadManager_DateDestinationAgentAssigned',
                'LeadManager_DateEstimatingAgentAssigned', 'LeadManager_DateSalesRepAssigned',
                'LeadManager_NextContactDate', 'LeadManager_FollowUpDate', 'LeadManager_LastUpdDateTime', 
                'LeadManager_LeadStartDate', 'ACM_StartDate', 'ACM_SurveyDate', 'Order_OrdSurveyDate',
                'Order_OrdRegistrationDate', 'Order_OrdHaulDecisionDate', 'Order_OrdStartDate',
                'Order_OrdLoadDate', 'Order_OrdDeliveryDate', 'Order_OrdSITODate', 'Order_OrdSITDDate'
]

def convert_columns_to_bool(dataframe, columns_to_convert):
    for column in columns_to_convert:
        dataframe[column].replace({'TRUE': True, 'FALSE': False}, inplace=True)
    return dataframe

boolean_columns = ['LeadManager_DistanceVerified', 'LeadManager_EnteredHot', 'LeadManager_Elevator', 
                   'LeadManager_Storage', 'LeadManager_DuplicateLead', 'LeadManager_AgentOwned',
                   'LeadManager_Locked', 'LeadManager_AgentFirstPricing', 'LeadManager_OriginAddressValidated',
                   'LeadManager_DestinationAddressValidated', 'LeadManager_DuplicateLock', 
                   'LeadManager_BookMoveRequest', 'LeadManager_InactiveDuplicate'
]

def convert_columns_to_float(dataframe, columns_to_convert):
    for column in columns_to_convert:
        dataframe[column] = dataframe[column].apply(pd.to_numeric, errors='coerce', downcast='float')
    return dataframe

float_columns = ['LeadManager_DiscountPercent', 'LeadManager_LeadCost', 'LeadManager_LeadSourceCost',
                 'Order_OrdRevValue']
# Convert columns to correct data types
crm_df['IsLeadFound'].replace({'Y': True, 'N': False}, inplace=True)
crm_df['IsShipmentBooked'].replace({'Y': True, 'N': False}, inplace=True)
crm_df['LeadManager_Appt'].replace({'Yes': True, 'No': False}, inplace=True)
convert_columns_to_int(crm_df, int_columns)
convert_columns_to_date(crm_df, date_columns)
convert_columns_to_bool(crm_df, boolean_columns)
convert_columns_to_float(crm_df, float_columns)
crm_df['ResidenceType'] = crm_df['LeadManager_CustomerNote'].str.extract(r'Residence Type:\s(.*?);', expand=False)

bin_edges = [x for x in range(0,5001,250)]
bin_labels = [f'{x} to {x+250}' for x in range(0,5000,250)]
crm_df['DistanceBin'] = pd.cut(crm_df['LeadManager_Distance'], bins=bin_edges, labels=bin_labels, right=False)

In [ ]:
# Get Website Behavior Data from GA4

In [ ]:
# Get GA4 data


ga4_df = pd.concat([july_df, aug_df, sep_df, oct_df, nov_df, dec_df, jan_df], ignore_index=True)

def convert_to_time(timestamp_microseconds):
    timestamp_microseconds = int(timestamp_microseconds)
    timestamp_seconds = timestamp_microseconds / 1000000
    regular_time = datetime.datetime.fromtimestamp(timestamp_seconds)
    return regular_time
ga4_df['event_timestamp'] = ga4_df['event_timestamp'].apply(convert_to_time)

# Create a mapping of 'ReferenceID' to 'user_pseudo_id'
mapping_df = ga4_df.dropna(subset=['ReferenceID'])[['user_pseudo_id', 'ReferenceID']]
# Find and assign ReferenceID values to instances of 'user_pseudo_id'
for _, row in mapping_df.iterrows():
    user_id = row['user_pseudo_id']
    ref_id = row['ReferenceID']
    ga4_df.loc[ga4_df['user_pseudo_id'] == user_id, 'ReferenceID'] = ref_id
    
booked_crm_df = crm_df[crm_df['IsShipmentBooked'] == True] # Find all booked rows
ga4_df['isBooked'] = ga4_df['ReferenceID'].isin(booked_crm_df['Leap_Reference_ID']).astype(bool) # assign booked to reference ids now that theyve been correctly assigned
    
ga4_df.sort_values(by=['event_timestamp', 'ReferenceID'])
print("DOPEr")

In [ ]:
ga4_df

# Condense GA4 Data and combine with CRM Data
## Not Ideal please do not Condense the data I could not figure out how to continue without condensing and it lead to an inaccurate model

In [ ]:
def condense_duplicate_rows(df):
    pseudo_grouped = df.groupby('user_pseudo_id')
    condensed_dfs = []

    # Go through each group
    for group_name, group_data in pseudo_grouped:
        # Create dict to store values for condensed row
        condensed_row = {}

        # Go through each column in group
        for col in df.columns:
            # Find the mode or first non-null value
            mode_or_non_null_value = group_data[col].mode().values[0] if group_data[col].count() > 0 else None
            # Add the value to the condensed row dict
            condensed_row[col] = mode_or_non_null_value
            
        # Sum EngagementTime_msec and store in TotalEngagementTime_msec col
        condensed_row['TotalEngagementTime_msec'] = group_data['EngagementTime_msec'].sum()
        condensed_dfs.append(pd.DataFrame([condensed_row]))
    # Concatenate the list of condensed DataFrames into a single DataFrame
    condensed_df = pd.concat(condensed_dfs, ignore_index=True)

    return condensed_df

condensed_ga4_df = condense_duplicate_rows(ga4_df)
combined_df = pd.merge(crm_df, condensed_ga4_df, left_on='Leap_Reference_ID', right_on='ReferenceID', how='inner')

In [ ]:
condensed_ga4_df

## Assign Columns of interest

In [ ]:
categorical_test_columns = [
    
    'ResidenceType',
    'SessionEngaged','PageTitle',
    'FormName','AAAMember','PageReferrer','SearchTerm',
    'mobile_brand_name','mobile_model_name','operating_system','operating_system_version',
    'language','browser_1','browser_version_1','hostname','city','country','continent','region','sub_continent','metro','name','medium','source',
    'platform','manual_campaign_id','manual_campaign_name','manual_source','manual_medium','manual_term','manual_content'
]

In [ ]:
# X = pd.get_dummies(combined_df[categorical_test_columns], drop_first=True)
# y = combined_df['IsShipmentBooked']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define the hyperparameter search space
# param_grid = {
#     'learning_rate': [0.01, 0.1],
#     'max_depth': [3, 4, 5],
#     'max_iter': [100, 200, 300]
# }

# # Create the GridSearchCV object
# grid_search = GridSearchCV(
#     estimator=HistGradientBoostingRegressor(),
#     param_grid=param_grid,
#     scoring='neg_mean_squared_error',  # Use the appropriate scoring metric
#     cv=5  # Number of cross-validation folds
# )

# # Perform the grid search
# grid_search.fit(X_train, y_train)

# # Get the best hyperparameters
# best_params = grid_search.best_params_

# # Train the final model with the best hyperparameters
# final_model = HistGradientBoostingRegressor(**best_params)
# final_model.fit(X_train, y_train)

# # Make predictions with the final model
# y_pred = final_model.predict(X_test)

In [ ]:
X = pd.get_dummies(combined_df[categorical_test_columns], drop_first=True)
y = combined_df['IsShipmentBooked']

# X.fillna(0, inplace=True)
# X.fillna(X.mode().iloc[0], inplace=True)

display(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = HistGradientBoostingRegressor()
# model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print('*(Ideal MSE == 0)*')
print(f"Mean Squared Error (MSE): {mse}")
print('\n')

rmse = np.sqrt(mse)
print('*(Ideal RMSE between 0.2 and 0.5)*')
print(f"Root Mean Squared Error (RMSE): {rmse}")
print('\n')

r2 = r2_score(y_test, y_pred)
print('*(Not really accurate but Ideally >=0.7)*')
print(f"R-squared (R²) Score: {r2}")
print('\n')

mae = mean_absolute_error(y_test, y_pred)
print('*(Ideal mae == 0)*')
print(f"Mean Absolute Error (MAE): {mae}")

# f1_score = f1_score(y_test, y_pred)
# print(f"F1 Score: {f1_score}")

print('\n')
roc_auc = roc_auc_score(y_test, y_pred)
print('*(Ideal ROC AUC == 1)*')
print(f"Area Under Receiver Operating Characteristics Score: {roc_auc}")

threshold = 0.5

y_train_binary = (y_train > threshold).astype(int)
y_test_binary = (y_test > threshold).astype(int)

# Create and fit a classification model
model = HistGradientBoostingClassifier()
model.fit(X_train, y_train_binary)

y_pred_binary = model.predict(X_test)

conf_matrix = confusion_matrix(y_test_binary, y_pred_binary)

print("Confusion Matrix:")
print(conf_matrix)
print('[[True Negative  False Positive]')
print('[False Negative  True Positive]]')

In [ ]:
combined_df

In [ ]:
combined_df['PredictionScore'] = model.predict(X)
true_df = combined_df[combined_df['IsShipmentBooked'] == True]
false_df = combined_df[combined_df['IsShipmentBooked'] != True]

In [ ]:
true_df['PredictionScore'].describe()

In [ ]:
false_df['PredictionScore'].describe()